In [1]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('key')
os.environ["KAGGLE_USERNAME"] = userdata.get('username')

In [2]:
!kaggle datasets download -d alfathterry/bbc-full-text-document-classification

Dataset URL: https://www.kaggle.com/datasets/alfathterry/bbc-full-text-document-classification
License(s): MIT
  0% 0.00/1.84M [00:00<?, ?B/s]
100% 1.84M/1.84M [00:00<00:00, 120MB/s]


In [3]:
!mkdir data

In [4]:
!unzip /content/bbc-full-text-document-classification.zip -d /content/data

Archive:  /content/bbc-full-text-document-classification.zip
  inflating: /content/data/bbc_data.csv  


In [5]:
import numpy as np
import gensim.downloader
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint, EarlyStopping
from tensorflow.keras import layers
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report
keras.utils.set_random_seed(42)

In [6]:
df=pd.read_csv("/content/data/bbc_data.csv",encoding='unicode_escape', on_bad_lines='skip')

In [7]:
df.head()

,data,labels
0,Musicians to tackle US red tape Musicians gro...,entertainment
1,"U2s desire to be number one U2, who have won ...",entertainment
2,Rocker Doherty in on-stage fight Rock singer ...,entertainment
3,Snicket tops US box office chart The film ada...,entertainment
4,"Oceans Twelve raids box office Oceans Twelve,...",entertainment


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    2225 non-null   object
 1   labels  2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [9]:
np.unique(df["labels"].values)

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [10]:
df.isnull().values.any()

False

In [11]:
fig = px.histogram(df, x=df['data'].apply(lambda x: x.split()).str.len().values ,nbins=50)
fig.update_layout(
    title="Sentences",
    xaxis_title="Sentences with number of words",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [12]:
fig = px.histogram(df, x=df['labels'] ,nbins=50)
fig.update_layout(
    title="Sentences",
    xaxis_title="labels",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

**Data preprocessing**

In [13]:
max_seq=450

In [14]:
df=df[df['data'].apply(lambda x: x.split()).str.len().values<=max_seq]

In [15]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
print(stopwords.words('english'))
stopWords=stopwords.words('english')

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [17]:
def remove(sentence,stopwords):
  newSentence=""
  for word in sentence.split():
    if(word not in stopwords):
      newSentence+=word+" "
  return newSentence

In [18]:
df['data']=df['data'].apply(lambda x: re.sub("[^0-9a-z\s]",'',x.lower())).apply(lambda sentence: " ".join([word for word in sentence.split() if word not in stopWords]))
df['encode']=df['labels'].astype('category').cat.codes.astype('int32')

In [19]:
df.head()

,data,labels,encode
0,musicians tackle us red tape musicians groups ...,entertainment,1
2,rocker doherty onstage fight rock singer pete ...,entertainment,1
3,snicket tops us box office chart film adaptati...,entertainment,1
4,oceans twelve raids box office oceans twelve c...,entertainment,1
5,landmark movies 2004 hailed us film profession...,entertainment,1


In [20]:
len(df)

1609

In [21]:
!pip install transformers==4.37.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 29.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [22]:
from transformers import BertTokenizerFast,TFBertModel

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [23]:
bert_model.config_class

transformers.models.bert.configuration_bert.BertConfig

In [24]:
data_tokens=tokenizer.batch_encode_plus(df['data'].tolist(), padding=True,return_tensors = 'np')

In [25]:
max_seq=data_tokens["input_ids"].shape[1]

In [26]:
labels=df['encode'].values

In [27]:
labels.shape

(1609,)

In [28]:
np.arange(labels.shape[0])

array([   0,    1,    2, ..., 1606, 1607, 1608])

In [29]:
xIdxTrain,xIdxTest,_,_=train_test_split(np.arange(labels.shape[0]), np.arange(labels.shape[0]), test_size=0.15, random_state=42)

In [30]:
xIdxTrain,xIdxval,_,_=train_test_split(xIdxTrain,xIdxTrain, test_size=0.1, random_state=42)

In [31]:
x_train_input=data_tokens['input_ids'][xIdxTrain]
x_train_attention=data_tokens['attention_mask'][xIdxTrain]
x_train_type=data_tokens['token_type_ids'][xIdxTrain]
y_train=labels[xIdxTrain]
x_val_input=data_tokens['input_ids'][xIdxval]
x_val_attention=data_tokens['attention_mask'][xIdxval]
x_val_type=data_tokens['token_type_ids'][xIdxval]
y_val=labels[xIdxval]
x_test_input=data_tokens['input_ids'][xIdxTest]
x_test_attention=data_tokens['attention_mask'][xIdxTest]
x_test_type=data_tokens['token_type_ids'][xIdxTest]
y_test=labels[xIdxTest]

In [32]:
print("x_train_input.shape= ",x_train_input.shape)
print(x_train_attention.shape)
print(x_train_type.shape)
print(y_train.shape)
print(x_test_input.shape)
print(x_test_attention.shape)
print(x_test_type.shape)
print(y_test.shape)

x_train_input.shape=  (1230, 415)
(1230, 415)
(1230, 415)
(1230,)
(242, 415)
(242, 415)
(242, 415)
(242,)


In [44]:
xx=tokenizer(df['data'].tolist()[0], padding=True,return_tensors = 'np')

In [46]:
bert_model(**xx,hidden_states=True)

ValueError: The following keyword arguments are not supported by this model: ['hidden_states'].

In [58]:
inputT=layers.Input(shape=(max_seq,),dtype = 'int32')
inputA=layers.Input(shape=(max_seq,),dtype = 'int32')
inputS=layers.Input(shape=(max_seq,),dtype = 'int32')
bert_output =bert_model(input_ids=inputT,attention_mask = inputA,token_type_ids=inputS,return_dict =True)
print(bert_output)
embedding = layers.Dropout(0.3)(bert_output["last_hidden_state"])
avg=layers.GlobalAveragePooling1D()(embedding)
print(avg)
dense=layers.Dense(256)(avg)
batch=layers.BatchNormalization()(dense)
act=layers.Activation('relu')(batch)
dense1=layers.Dense(5)(act)
batch1=layers.BatchNormalization()(dense1)
output=layers.Activation('softmax')(batch1)
model=keras.Model([inputT,inputA,inputS],output)

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 415, 768) dtype=float32 (created by layer 'tf_bert_model')>, pooler_output=<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>, past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='global_average_pooling1d/Mean:0', description="created by layer 'global_average_pooling1d'")


In [59]:
print(model.layers[3])
model.layers[3].trainable=False
print(model.layers[3].trainable)

False


In [50]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 415)]                0         []                            
                                                                                                  
 input_8 (InputLayer)        [(None, 415)]                0         []                            
                                                                                                  
 input_9 (InputLayer)        [(None, 415)]                0         []                            
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_7[0][0]',             
 )                           ngAndCrossAttentions(last_   40         'input_8[0][0]',       

In [60]:
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [37]:
y_val.shape

(137,)

In [38]:
y_train.shape

(1230,)

In [39]:
x_train_input.shape

(1230, 415)

In [62]:
model.fit([x_train_input,x_train_attention,x_train_type], y_train, epochs=20, batch_size=128  ,
          validation_data=([x_val_input,x_val_attention,x_val_type],y_val ) ,  verbose = 1)

Epoch 1/20
10/10 [==============================] - 51s 5s/step - loss: 0.4741 - accuracy: 0.9691 - val_loss: 1.1771 - val_accuracy: 0.6861
Epoch 2/20
10/10 [==============================] - 47s 5s/step - loss: 0.4302 - accuracy: 0.9740 - val_loss: 1.0893 - val_accuracy: 0.7883
Epoch 3/20
10/10 [==============================] - 47s 5s/step - loss: 0.4017 - accuracy: 0.9805 - val_loss: 1.0584 - val_accuracy: 0.8905
Epoch 4/20
10/10 [==============================] - 47s 5s/step - loss: 0.3847 - accuracy: 0.9846 - val_loss: 1.0493 - val_accuracy: 0.9197
Epoch 5/20
10/10 [==============================] - 47s 5s/step - loss: 0.3662 - accuracy: 0.9878 - val_loss: 1.0412 - val_accuracy: 0.9270
Epoch 6/20
10/10 [==============================] - 47s 5s/step - loss: 0.3528 - accuracy: 0.9894 - val_loss: 1.0331 - val_accuracy: 0.9416
Epoch 7/20
10/10 [==============================] - 47s 5s/step - loss: 0.3418 - accuracy: 0.9911 - val_loss: 1.0281 - val_accuracy: 0.9416
Epoch 8/20
10/10 [==

In [73]:
y_predict=model.predict([x_test_input,x_test_attention,x_test_type])

8/8 [==============================] - 7s 918ms/step


In [74]:
y_predict

array([[0.14963359, 0.5467403 , 0.09063472, 0.12100185, 0.09198953],
       [0.14771806, 0.488911  , 0.11194208, 0.14495365, 0.10647526],
       [0.5611559 , 0.11618675, 0.09542826, 0.13042559, 0.09680356],
       ...,
       [0.35091123, 0.14224006, 0.13643552, 0.17650996, 0.19390324],
       [0.13216524, 0.11014524, 0.10855113, 0.5710775 , 0.07806084],
       [0.2784594 , 0.20193875, 0.21708225, 0.1816887 , 0.12083086]],
      dtype=float32)

In [75]:
y_predict=np.argmax(y_predict,axis=-1)

In [76]:
y_predict.shape

(242,)

In [77]:
y_test.shape

(242,)

In [79]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96        77
           1       1.00      1.00      1.00        51
           2       0.97      0.91      0.94        34
           3       1.00      1.00      1.00        54
           4       1.00      0.88      0.94        26

    accuracy                           0.97       242
   macro avg       0.98      0.96      0.97       242
weighted avg       0.97      0.97      0.97       242

